In [8]:
# Import the dependencies
import pandas as pd
import os
import numpy as np
import datetime 
import time
from calendar import timegm

In [9]:
# File to Load 
supply_data_to_load = "ETL/CAISO-supply-20220316.csv"

In [20]:
# Read the CSV and store into a Pandas DataFrame
supply_data_df = pd.read_csv(supply_data_to_load)
supply_data_df.head()

,Supply 03/16/2022,0:00,0:05,0:10,0:15,0:20,0:25,0:30,0:35,0:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,Unnamed: 289
0,Renewables,6387,6514,6562,6575,6546,6512,6498,6325,6220,...,5658,5585,5515,5465,5429,5387,5340,5323,5245,NaN
1,Natural gas,5353,5197,5060,5024,4928,4858,4790,4805,4814,...,6337,6305,6119,6062,5915,5842,5848,5726,5724,NaN
2,Large hydro,1087,1004,869,872,873,875,873,888,911,...,1305,1351,1323,1316,1292,1277,1277,1247,1228,NaN
3,Imports,7819,7857,8022,8017,8051,8062,8010,7935,7830,...,8089,8070,8043,8091,8128,8156,8168,8255,8238,NaN
4,Batteries,-88,-3,-8,-18,21,-9,29,-4,6,...,95,86,97,27,88,59,53,-5,103,NaN


In [26]:
colname = supply_data_df.columns[0]
date = colname.split(" ")[1]
date

'03/16/2022'

In [19]:
# Reset index to energy sources and transpose
supply_df_transpose = supply_data_df.set_index('Supply 03/16/2022').T
supply_df_transpose.head()

Supply 03/16/2022,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other
0:00,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0
0:05,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0
0:10,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0
0:15,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0
0:20,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0


In [27]:
# Create date column
supply_df_transpose["Date"] = date
supply_df_transpose.head()

Supply 03/16/2022,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date
0:00,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,03/16/2022
0:05,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,03/16/2022
0:10,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,03/16/2022
0:15,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,03/16/2022
0:20,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,03/16/2022


In [ ]:
# Create time column 
supply_df_transpose["Time"] = supply_df_transpose.index
supply_df_transpose

In [ ]:
# Reset index to start from 0 and remove index name
supply_df_transpose.reset_index(drop=True, inplace=True)
supply_df_transpose.rename_axis(None, axis=1, inplace=True)
supply_df_transpose.head()

In [ ]:
# Cast 'Time' column to string and append seconds
supply_df_transpose['Time'] = supply_df_transpose['Time'].apply(str) + ':00'
supply_df_transpose.iloc[0:120, 9] = "0" + supply_df_transpose.iloc[0:120, 9]
supply_df_transpose.head(120)

In [ ]:
# Add 'Date' and 'Time' columns together
supply_df_transpose['DateTime'] = supply_df_transpose['Date'] + 'T' + supply_df_transpose['Time']
supply_df_transpose.head()

In [ ]:
supply_df_transpose = supply_df_transpose.drop([288, 0])

In [ ]:
supply_df_transpose['DateTime'] =  pd.to_datetime(supply_df_transpose['DateTime'], format='%Y-%m-%dT%H:%M:%S.%f')
supply_df_transpose.dtypes

In [ ]:
supply_df_transpose['Unix Timestamp'] = supply_df_transpose.DateTime.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())

In [ ]:
supply_df_transpose.head()

In [ ]:
supply_df_transpose['Unix Timestamp'].astype(np.int64)

In [ ]:
# datetime.fromisoformat('2022-03-16T23:55:00')

In [ ]:
# ts= int(datetime.datetime(2022, 3, 16, 23, 55).timestamp())
# print(ts)